# Blue Brain BioExplorer - Neuromodulation
![](../bioexplorer_neuromodulation_banner.png)

## Visualization of a microtubule

In [ ]:
from bioexplorer import BioExplorer, Volume, Protein, AnimationParams, Vector3, Quaternion

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
obj_folder = resource_folder + 'obj/'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
transporters_folder = pdb_folder + 'transporters/'
tubules_folder = pdb_folder + 'tubules/'

representation = be.REPRESENTATION_ATOMS_AND_STICKS

scale = 1000.0

In [ ]:
import numpy as np
import quaternion
import math
from bioexplorer import MovieMaker

index = 4

def createFromAxisAngle(axis, angle):
    halfAngle = angle * 0.5
    s = math.sin(halfAngle)
    return quaternion.quaternion(axis[0] * s, axis[1] * s, axis[2] * s, math.cos(halfAngle))


def lookAt(sourcePoint, destPoint):
    forward = np.array([0.0, 0.0, 1.0])
    forwardVector = np.subtract(destPoint, sourcePoint)
    forwardVector = forwardVector / np.sqrt(np.sum(forwardVector**2))
    dot = np.dot(forward, forwardVector)
    angle = math.acos(dot)
    axis = np.cross(forward, forwardVector)
    axis = axis / np.sqrt(np.sum(axis**2))
    return createFromAxisAngle(axis, angle)

points = be.get_neuron_section_points('Neuron', 4, 0)
p0 = points[index]
p1 = points[index + 1]
print(p0)
print(p1)

src = np.array(p0[:3])
dst = np.array(p1[:3])
q = lookAt(src, dst)
Q = Quaternion(q.w, q.x, q.y, q.z)
print(Q)

mm = MovieMaker(be)
forwardVector = np.subtract(dst, src)
forward_length = np.sqrt(np.sum(forwardVector**2))
mm.set_camera(origin=p0, direction=list(forwardVector[:3]), up=[0,1,0])
core.set_renderer()


In [ ]:
# be.add_sphere(name='test', position=Vector3(p0[0], p0[1], p0[2]), radius=0.7 * scale, color=Vector3(1,1,1))
# be.add_sphere(name='test', position=Vector3(p1[0], p1[1], p1[2]), radius=0.7 * scale, color=Vector3(1,1,1))

In [ ]:
import random

size  = Vector3(5.0, forward_length, 0.0)
for i in range(5):
    position = Vector3(
        p0[0] + random.randint(-scale / 20.0, scale / 20.0),
        p0[1] + random.randint(-scale / 20.0, scale / 20.0),
        p0[2] + random.randint(-scale / 20.0, scale / 20.0))
    name = 'Microtubule%05d' % i

    # ACE2 receptor definition
    tubule = Protein(
        name=name + '_TUB',
        source=tubules_folder + '1tub.pdb', 
        rotation=Quaternion(0.0, 0.0, 0.707, 0.707),
        occurences=size.y * 5.0)

    volume = Volume(
        name=name,
        shape=be.ASSEMBLY_SHAPE_HELIX, shape_params=size,
        protein=tubule)

    status = be.add_volume(
        position=position,
        rotation=Quaternion( 0.212,    0.700,   -0.231,    0.642),
        volume=volume)

    status = be.set_protein_color_scheme(
        assembly_name=name,
        name=name + '_TUB',
        color_scheme=be.COLOR_SCHEME_CHAINS,
        palette_name='Set2', palette_size=5)
core.set_renderer()

In [ ]:
be.reset_camera()
core.set_renderer()

In [ ]:
core.set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[27451.090107478147, 195917.513949172, 38855.97730085014],
    target=[27451.090107478147, 195917.513949172, 38338.56432876815],    
)

## Rendering settings

In [ ]:
status = be.reset_camera()
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
mm.create_snapshot(
    renderer='bio_explorer',
    path='/tmp', base_name='neuromodulation_4k_v1',
    size=[4 * 960, 4 * 540], samples_per_pixel=64)